<a href="https://colab.research.google.com/github/Bilal0335/AI-Agent/blob/main/composio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio whisper gtts google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install requests
!pip install openai-whisper groq


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━

In [ ]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib whisper gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 72.2 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.137.0
    Uninstalling google-api-python-client-2.137.0:
      Successfully uninstalled google-api-python-client-2.137.0


In [ ]:
import os
import gradio as gr
import whisper
import datetime
import re
import requests
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# Load the Whisper model
model = whisper.load_model("base")

# Google Calendar API setup
SCOPES = ['https://www.googleapis.com/auth/calendar']

def authenticate_google_calendar():
    """Authenticate and return a Google Calendar service object."""
    creds = None
    if os.path.exists('token.json'):
        print("Found token.json, using it for authentication.")
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            print("Refreshing expired token.")
            creds.refresh(Request())
        else:
            print("No valid credentials, running OAuth flow.")
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('calendar', 'v3', credentials=creds)

def add_event_to_google_calendar(summary, description, start_time, end_time):
    try:
        service = authenticate_google_calendar()
        event = {
            'summary': summary,
            'description': description,
            'start': {
                'dateTime': start_time.isoformat(),
                'timeZone': 'UTC',
            },
            'end': {
                'dateTime': end_time.isoformat(),
                'timeZone': 'UTC',
            },
        }
        print(f"Attempting to add event: {event}")
        event_result = service.events().insert(calendarId='primary', body=event).execute()
        print(f"Event created: {event_result.get('htmlLink')}")
        return f"Event created: {event_result.get('htmlLink')}"
    except Exception as e:
        print(f"Error creating event: {e}")
        return f"An error occurred: {e}"

def extract_dates(text):
    """Extract start and end times from text in the format '9:00 am - 10:00 am'."""
    pattern = r'(\d{1,2}:\d{2} [ap]m) - (\d{1,2}:\d{2} [ap]m)'
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        start_time_str, end_time_str = match.groups()
        now = datetime.datetime.utcnow()
        try:
            start_time = datetime.datetime.strptime(start_time_str, "%I:%M %p").replace(year=now.year, month=now.month, day=now.day)
            end_time = datetime.datetime.strptime(end_time_str, "%I:%M %p").replace(year=now.year, month=now.month, day=now.day)
            print(f"Extracted start time: {start_time}, end time: {end_time}")
            return start_time, end_time
        except ValueError as ve:
            print(f"Date extraction error: {ve}")
            return None, None
    print("No date match found in text.")
    return None, None

def process_input(audio_file, text_input):
    try:
        transcription_text = ""
        if audio_file:
            # Process audio file
            audio = whisper.load_audio(audio_file)
            result = model.transcribe(audio)
            text = result['text']
            transcription_text = text  # Store transcription for display
            print(f"Transcription: {transcription_text}")
        elif text_input:
            # Use text input directly
            text = text_input
        else:
            return "No input provided", None, None

        # Refine the prompt for concise responses
        refined_prompt = (
            f"Please provide a concise schedule for the following input: '{text}'. "
            f"Include only start and end times."
        )

        # For demonstration purposes, the prompt is not being sent to Groq (since API setup not provided)
        response_message = refined_prompt  # This can be a dynamic response from a model

        # Extract start and end times from the response
        start_time, end_time = extract_dates(response_message)

        if start_time and end_time:
            summary = "Scheduled Task"
            description = response_message

            # Add event to Google Calendar
            calendar_response = add_event_to_google_calendar(summary, description, start_time, end_time)
            return transcription_text, response_message, calendar_response
        else:
            return transcription_text, "Could not extract start and end times from the response", None

    except Exception as e:
        print(f"An error occurred: {e}")
        return f"An error occurred: {e}", None, None

iface = gr.Interface(
    fn=process_input,
    inputs=[
        gr.Audio(type="filepath", label="Upload Audio File (optional)"),
        gr.Textbox(placeholder="Or type your text here...", label="Text Input (optional)")
    ],
    outputs=[
        gr.Textbox(label="Transcription Text"),  # Show transcription text here
        gr.Textbox(label="Response Text"),
        gr.Textbox(label="Google Calendar Response")
    ],
    live=True,
    title="AI Daily Productivity Planner",
    description="Upload an audio file or type text input. The AI planner will transcribe or process it, extract start and end times, and update Google Calendar."
)

iface.launch()


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ec07d8fc7c40817fd8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
